<img src="https://cienciasgeodesicas.ufpr.br/wp-content/uploads/2021/03/cropped-folder-site3.png" alt="Drawing"/>

**Federal University of Parana, Curitiba, Brazil**

Geodetic Science Graduate Programme

---

Authors:
 - Darlan Miranda Nunes | [ORCID- 0000-0003-3557-5341](https://orcid.org/0000-0001-5566-7919)
 - Silvana Philippi Camboim | [ORCID- 0000-0003-3557-5341](https://orcid.org/0000-0003-3557-5341)
---

## Collaborative Toponyms in OpenStreetMap: an open-source framework to investigate the relationship with intrinsic quality parameters

DOI: [https://doi.org/10.1080/15230406.2025.2566807](https://doi.org/10.1080/15230406.2025.2566807)

# Jupyter notebook 02: Analysis of Collaborative Toponyms in OpenStreetMap

## Install the necessary libraries to the analysis

In [ ]:
# Just in case of using colab, install these necessary libraries
# %pip install geopandas matplotlib pysal seaborn mapclassify mgwr -q
!pip install --upgrade selenium
!pip install --upgrade typing_extensions

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File 

## Import the libraries

In [1]:
# Import library and some pre-installed modules
import os
import sys
import numpy as np
import json
import folium
import pandas as pd
import geopandas as gpd
import mapclassify
import matplotlib.colors as mcolors
import seaborn as sns
import statsmodels.api as sm
import warnings
import pyproj
import branca.colormap as cm
from matplotlib import pyplot as plt
import matplotlib.cm as colormaps
from ipywidgets import widgets, Layout, Button, interact, Dropdown, SelectMultiple, HBox, VBox, Output
from IPython.display import display, clear_output, FileLink
from folium import plugins, Map, Element, Figure, LayerControl, TileLayer
from folium.features import GeoJson, GeoJsonTooltip,Choropleth, CircleMarker
from pysal.explore import esda
from pysal.lib import weights
from shapely.geometry import MultiPolygon, box
from shapely.ops import transform
from jinja2 import Template
from tqdm.notebook import tqdm
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from esda import G_Local
from esda.moran import Moran
from splot.esda import plot_moran
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

/usr/local/lib/python3.8/dist-packages/spaghetti/network.py:36: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(f"{dep_msg}", FutureWarning)


In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [3]:
# Check the current working directory
os.getcwd()

'/data/private/Data_Challenge/replication'

In [83]:
# Check the current directories in the working directory
%ls

data/       results/                            results_amst_centrum_100m_full/
notebooks/  results_ams_centwest_100m_trimmed/


## Analysis of Collaborative Toponyms in OpenStreetMap

### Import the regular grid cells with the aggregated data extracted from OSM via the OHSOME API
 * Results from the Algorithm 1 implemented in Jupyter Notebook 1 (JN1)

In [84]:
# Import the grid with the aggregated data extracted from OSM via the OHSOME API
grid_osm = None

# Output to display messages
output = widgets.Output()

# Base path for the directories with GeoJSON files
base_path = 'data/output_code1'

# Store the name of the selected GeoJSON file
selected_file = None

# Function for selecting and loading the GeoJSON file
def select_file(change):
    global grid_osm, selected_file
    selected_file = change['new']
    
    if selected_file != "Select the GeoJSON file of the case study area:":
        file_path = os.path.join(base_path, selected_file)
        try:
            with open(file_path, 'r') as file:
                grid_osm = json.load(file)
            with output:
                clear_output(wait=True)
                print("File selected: %s loaded with success!" %(selected_file))
                print("\nFile path:", file_path)
        except FileNotFoundError:
            with output:
                clear_output(wait=True)
                print("File not found:", selected_file)

# Listing available GeoJSON files
file_list = [f for f in os.listdir(base_path) if f.endswith('.geojson')]
options = ["Select the GeoJSON file of the case study area:"] + file_list

# Dropdown to select the GeoJSON file
dropdown = widgets.Dropdown(options=options)
dropdown.observe(select_file, names='value')

# Display the dropdown
display(dropdown, output)

Dropdown(options=('Select the GeoJSON file of the case study area:', 'amsterdam_centrum_100m_trimmed_results.g…

Output()

### Convert GeoJSON Data to GeoDataFrame

In [85]:
# Convert GeoJSON Data to GeoDataFrame
gdf_osm = gpd.GeoDataFrame.from_features(grid_osm['features'])

In [86]:
# Check the five first records of GeoDataFrame
gdf_osm.head()

,geometry,id,left,top,right,bottom,row_index,col_index,zero_feature_cell,leisure_total_count,...,building_contrib_ratio,leisure_users_total,leisure_users_named,leisure_users_ratio,amenity_users_total,amenity_users_named,amenity_users_ratio,building_users_total,building_users_named,building_users_ratio
0,"POLYGON ((4.87326 52.37246, 4.87619 52.37247, ...",10,120000.0001,487300.0005,120200.0001,487100.0005,9,0,False,4.0,...,2.242152,8.0,0.0,0.000000,26.0,17.0,65.384615,18.0,4.0,22.222222
1,"POLYGON ((4.87613 52.37786, 4.87907 52.37788, ...",21,120200.0001,487900.0005,120400.0001,487700.0005,6,1,False,5.0,...,7.569721,17.0,10.0,58.823529,18.0,10.0,55.555556,26.0,14.0,53.846154
2,"POLYGON ((4.87615 52.37607, 4.87909 52.37608, ...",22,120200.0001,487700.0005,120400.0001,487500.0005,7,1,False,1.0,...,0.000000,1.0,1.0,100.000000,15.0,12.0,80.000000,12.0,0.0,0.000000
3,"POLYGON ((4.87617 52.37427, 4.87911 52.37428, ...",23,120200.0001,487500.0005,120400.0001,487300.0005,8,1,False,3.0,...,0.579710,2.0,1.0,50.000000,24.0,21.0,87.500000,15.0,2.0,13.333333
4,"POLYGON ((4.87619 52.37247, 4.87913 52.37248, ...",24,120200.0001,487300.0005,120400.0001,487100.0005,9,1,False,0.0,...,1.023891,0.0,0.0,0.000000,21.0,19.0,90.476190,13.0,2.0,15.384615


In [87]:
# Check the number os grid cell with information retrived of OSM
len(gdf_osm)

88

In [88]:
print("CRS is None?", gdf_osm.crs)

if gdf_osm.crs is None:
    gdf_osm = gdf_osm.set_crs(epsg=4326)

print("\nCRS defined:")
gdf_osm.crs

CRS is None? None

CRS defined:


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

---
### Quantitative Analysis

#### 1. Preliminary Exploratory analysis

##### 1.1 Histogram Analysis

In [89]:
# Preliminary analysis with histograms - one plot version

# === Diretório de saída ===
histogram_output_dir = 'results/1_descriptive_statistics/1_histograms/'
os.makedirs(histogram_output_dir, exist_ok=True)

# === Função para extrair nome da área ===
def extract_study_area_name(selected_file):
    study_area_name, _ = os.path.splitext(selected_file)
    tokens = study_area_name.replace("-", "_").split("_")

    try:
        idx_results = tokens.index("results")
        study_area_tokens = tokens[:idx_results - 2]
    except ValueError:
        # fallback if 'results' is missing
        study_area_tokens = tokens[:-2]

    study_area_name_only = (
        " ".join(study_area_tokens)
        .replace("-", " ")
        .replace("_", " ")
        .title()
    )
    return study_area_name_only
# === Nome da área ===
study_area_name_only = extract_study_area_name(selected_file)

# === Configurações das variáveis ===
feature_order = ["leisure_name_ratio", "building_name_ratio", "amenity_name_ratio"]
feature_labels = {
    "leisure_name_ratio": "Leisure",
    "building_name_ratio": "Building",
    "amenity_name_ratio": "Amenity"
}
feature_colors = {
    "leisure_name_ratio": "#4e79a7",   # azul
    "building_name_ratio": "#e15759",  # vermelho
    "amenity_name_ratio": "#59a14f",   # verde
}

# === Widgets ===
plot_button = widgets.Button(description='Generate Histogram', button_style='success')
save_button = widgets.Button(description='Save Plot as PNG', button_style='info', disabled=True)
output = widgets.Output()

# === Função para plotar histogramas lado a lado + KDE ===
def plot_grouped_histogram_kde(b=None):
    clear_output(wait=True)
    display(widgets.VBox([plot_button, save_button, output]))

    with output:
        output.clear_output()
        fig, ax = plt.subplots(figsize=(8, 5))
        #ax2 = ax.twinx()  # eixo Y para o KDE

        bin_edges = np.arange(0, 101, 5) # bins de 0 a 100, largura 5
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        bar_width = 1.5  # Largura de cada barra
        total_groups = len(feature_order)

        for i, col in enumerate(feature_order):
            values = gdf_osm[col].dropna()
            counts, _ = np.histogram(values, bins=bin_edges)

            # Calcula deslocamento para desenhar barras lado a lado
            #offset = (i - total_groups / 2) * bar_width + bar_width / 2
            offset = (i - (total_groups - 1) / 2) * bar_width
            ax.bar(bin_centers + offset, counts,
                    width=bar_width, 
                    color=feature_colors[col],
                    alpha=0.9,
                    label=feature_labels[col],
                    edgecolor='gray',
                    linewidth=0.3)

            # KDE curva
            sns.kdeplot(values,
                        ax=ax,
                        color=feature_colors[col],
                        linewidth=2,
                        bw_adjust=1.2,
                        clip=(0, 100),
                        cut=0)

        # Estilo
        plt.title(f"Histogram of name-tag coverage in {study_area_name_only}", fontsize=16, pad=20)
        plt.xlabel("Grid cell values of name ratio (%)", fontsize=16, labelpad=10)
        plt.ylabel("Frequency", fontsize=16, labelpad=10)
        plt.xticks(np.arange(0, 101, 20), fontsize=14)
        plt.yticks(np.arange(0, 201, 20), fontsize=14)
        plt.xlim(0, 100)
        plt.ylim(0, 200)
        plt.grid(False)
        #ax.tick_params(axis='both', labelsize=14)
        plt.legend(title="OSM tag", fontsize=12, title_fontsize=13)
        plt.tight_layout()
        plt.show()
        save_button.disabled = False

# === Função para salvar ===
def save_grouped_histogram_kde(b):
    save_path = os.path.join(
        histogram_output_dir,
        f'grouped_histogram_kde_leisure_building_amenity_{study_area_name_only.replace(" ", "_").lower()}.png'
    )
    fig, ax = plt.subplots(figsize=(8, 5))

    bin_edges = np.arange(0, 101, 5)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bar_width = 1.5
    total_groups = len(feature_order)

    for i, col in enumerate(feature_order):
        values = gdf_osm[col].dropna()
        counts, _ = np.histogram(values, bins=bin_edges)

        offset = (i - (total_groups - 1) / 2) * bar_width
        ax.bar(bin_centers + offset, counts,
                width=bar_width,
                color=feature_colors[col],
                alpha=0.9,
                label=feature_labels[col],
                edgecolor='gray',
                linewidth=0.3)
        
        sns.kdeplot(values,
                    ax=ax,
                    color=feature_colors[col],
                    linewidth=2,
                    bw_adjust=1.2,
                    clip=(0, 100),
                    cut=0)

    plt.title(f"Histogram of name-tag coverage in {study_area_name_only}", fontsize=16, pad=15)
    plt.xlabel("Grid cell values of name ratio (%)", fontsize=16, labelpad=10)
    plt.ylabel("Frequency", fontsize=16, labelpad=10)
    plt.xticks(np.arange(0, 101, 20), fontsize=14)
    plt.yticks(np.arange(0, 201, 20), fontsize=14)
    plt.xlim(0, 100)
    plt.ylim(0, 200)
    plt.grid(False)
    plt.legend(title="OSM tag", fontsize=12, title_fontsize=13)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    with output:
        print(f"Plot saved at: {save_path}")

# === Conexão dos botões ===
plot_button.on_click(plot_grouped_histogram_kde)
save_button.on_click(save_grouped_histogram_kde)
display(widgets.VBox([plot_button, save_button, output]))

# Gerar gráfico inicial
plot_grouped_histogram_kde()

##### 1.2 BoxPlot Analysis

In [90]:
# Preliminary analysis with boxplots

# Setup directory for saving boxplots
boxplot_output_dir = 'results/1_descriptive_statistics/2_boxplots/'
os.makedirs(boxplot_output_dir, exist_ok=True)

# Ensure that selected_file is already defined elsewhere
study_area_name_only = extract_study_area_name(selected_file)

# --- Widgets ---
dropdown_boxplot = widgets.Dropdown(
    options=[col for col in gdf_osm.columns if col != 'geometry'],
    value='leisure_name_ratio',
    description='Column:',
    disabled=False
)

plot_boxplot_button = widgets.Button(
    description='Generate Boxplot',
    button_style='success'
)

save_boxplot_button = widgets.Button(
    description='Save Boxplot as PNG',
    button_style='info',
    disabled=True
)

output_boxplot = widgets.Output()

# --- Functions ---
def plot_boxplot(b=None):
    clear_output(wait=True)
    display(widgets.VBox([dropdown_boxplot, plot_boxplot_button, save_boxplot_button, output_boxplot]))

    column = dropdown_boxplot.value
    with output_boxplot:
        output_boxplot.clear_output()
        plt.figure(figsize=(6, 6))
        sns.boxplot(y=gdf_osm[column].dropna())
        plt.title(f'Boxplot of {column} of {study_area_name_only}', fontsize=14)
        plt.ylabel(f'{column} values')
        plt.grid(False)
        plt.tight_layout()
        plt.show()

        save_boxplot_button.disabled = False

def save_boxplot(b):
    column = dropdown_boxplot.value
    save_path = os.path.join(
        boxplot_output_dir,
        f'boxplot_{column.lower()}_{study_area_name_only.replace(" ", "_").lower()}.png'
    )
    plt.figure(figsize=(6, 6))
    sns.boxplot(y=gdf_osm[column].dropna())
    plt.title(f'Boxplot of {column} of {study_area_name_only}', fontsize=14)
    plt.ylabel(f'{column} values')
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    with output_boxplot:
        print(f"Boxplot saved at: {save_path}")

# --- Bind Buttons ---
plot_boxplot_button.on_click(plot_boxplot)
save_boxplot_button.on_click(save_boxplot)

# --- Display ---
display(widgets.VBox([dropdown_boxplot, plot_boxplot_button, save_boxplot_button, output_boxplot]))

# Generate the initial plot
plot_boxplot()

#### 2. Ordinary Least Squares (OLS) regression

In [91]:
# Convert GeoDataFrame to DataFrame by removing the geometry column
gdf_osm_copy = gdf_osm.copy()
df_osm = gdf_osm_copy.drop(columns=['geometry'])

In [92]:
# Check the five first records of DataFrame
df_osm.head()

,id,left,top,right,bottom,row_index,col_index,zero_feature_cell,leisure_total_count,leisure_name_count,...,building_contrib_ratio,leisure_users_total,leisure_users_named,leisure_users_ratio,amenity_users_total,amenity_users_named,amenity_users_ratio,building_users_total,building_users_named,building_users_ratio
0,10,120000.0001,487300.0005,120200.0001,487100.0005,9,0,False,4.0,0.0,...,2.242152,8.0,0.0,0.000000,26.0,17.0,65.384615,18.0,4.0,22.222222
1,21,120200.0001,487900.0005,120400.0001,487700.0005,6,1,False,5.0,2.0,...,7.569721,17.0,10.0,58.823529,18.0,10.0,55.555556,26.0,14.0,53.846154
2,22,120200.0001,487700.0005,120400.0001,487500.0005,7,1,False,1.0,1.0,...,0.000000,1.0,1.0,100.000000,15.0,12.0,80.000000,12.0,0.0,0.000000
3,23,120200.0001,487500.0005,120400.0001,487300.0005,8,1,False,3.0,1.0,...,0.579710,2.0,1.0,50.000000,24.0,21.0,87.500000,15.0,2.0,13.333333
4,24,120200.0001,487300.0005,120400.0001,487100.0005,9,1,False,0.0,0.0,...,1.023891,0.0,0.0,0.000000,21.0,19.0,90.476190,13.0,2.0,15.384615


In [93]:
# Check the data types
df_osm.dtypes

id                                         int64
left                                     float64
top                                      float64
right                                    float64
bottom                                   float64
row_index                                  int64
col_index                                  int64
zero_feature_cell                           bool
leisure_total_count                      float64
leisure_name_count                       float64
leisure_name_ratio                       float64
amenity_total_count                      float64
amenity_name_count                       float64
amenity_name_ratio                       float64
building_total_count                     float64
building_name_count                      float64
building_name_ratio                      float64
leisure_latest5_name_contributions       float64
building_latest5_name_contributions      float64
amenity_latest5_name_contributions       float64
leisure_name_tagChan

In [94]:
# Perform Multiple Linear Regression (OLS)

# Setup directory for saving OLS results
ols_output_dir = 'results/2_OLS_regression'
os.makedirs(ols_output_dir, exist_ok=True)

# Dynamic selection of the dependent variable and the independent variables
# Dependent variables
dependent_var_widget = Dropdown(options=[col for col in df_osm.columns if col != 'id'],
                                description='Dependent Var:',value = 'leisure_name_ratio')

# Independent variables
independent_vars_widget = SelectMultiple(
    options=[col for col in df_osm.columns if col != 'id'],
    description='Independent Vars (use ctrl to select multiple variables):',
    layout={'width': '90%', 'height': '200px'})

# Global Dictionary to store all regression summaries
regression_summaries = {}

# Global dictionary for storing R² values
r2_values = {}

# Function to run the regression
def run_regression(button):
    global regression_summaries, r2_values
    dependent_var = dependent_var_widget.value
    independent_vars = list(independent_vars_widget.value)
    X = df_osm[independent_vars]
    y = df_osm[dependent_var]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    # Print the summary for the current run
    display(model.summary())

    # To save the summary as text file
    regression_summary = model.summary().as_text()

    # Store the summary using the model name as the key
    model_name = f"{dependent_var} ~ {' + '.join(independent_vars)}"
    regression_summaries[model_name] = regression_summary

    # Store R² Adjusted values
    r2_values[model_name] = model.rsquared_adj

# Function to clear the outputs
def clear_outputs(button):
    clear_output()
    display_widgets()

# Function to save all regression results
def save_regression_results(button):
    study_area_name_only = extract_study_area_name(selected_file)
    detected_tag = dependent_var_widget.value.split('_')[0] if '_' in dependent_var_widget.value else 'unknown'
    save_filename = f'regression_summaries_{detected_tag}_{study_area_name_only.replace(" ", "_")}.txt'
    save_path = os.path.join(ols_output_dir, save_filename)

    if regression_summaries:
        with open(save_path, "w") as file:
            for model, summary in regression_summaries.items():
                file.write(f"Model: {model}\n")
                file.write(f"{summary}\n\n")
        print(f"All results saved successfully at: {save_path}")
    else:
        print("No results to save.")

# Function to Buttons actions
def button_actions(b):
    if b.description == 'Run Regression':
        run_regression(b)
    elif b.description == 'Clear Outputs':
        clear_outputs(b)
    elif b.description == 'Save Results':
        save_regression_results(b)

# Create Buttons to run the analysis
run_button = Button(description='Run Regression',
                    button_style='success')
save_button = Button(description="Save Results",
                     button_style='info')
clear_button = Button(description="Clear Outputs",
                      button_style='danger')

# Assign the button_actions function as the on_click event handler for all buttons
for button in [run_button, save_button, clear_button]:
    button.on_click(button_actions)

# Organize the widgets
variable_selection_box = VBox([dependent_var_widget, independent_vars_widget])
buttons_box = VBox([run_button, save_button, clear_button])

# Function to display the widgets
def display_widgets():
    hbox = HBox([variable_selection_box, buttons_box])
    display(hbox)

# Display the Widgets and the Buttons
display_widgets()

---
### Spatial Patterns Analysis

#### 3. Choropleth maps

In [97]:
# Make Choropleth Maps

# Helper: calculate centroid
def calculate_centroid_of_union(gdf):
    union_poly = gdf.geometry.unary_union
    if isinstance(union_poly, MultiPolygon):
        union_poly = MultiPolygon(union_poly).convex_hull
    return [union_poly.centroid.y, union_poly.centroid.x]

# Add choropleth with manual styling
def add_choropleth_manual(map_obj, gdf, column, method='Quantiles', palette='YlGn'):
    unique_values = len(gdf[column].unique())
    k = min(4, unique_values)

    # Compute classes
    if method == 'Quantiles':
        classifier = mapclassify.Quantiles(gdf[column], k=k)
        class_labels = pd.Series(classifier.yb, index=gdf.index)
    elif method == 'EqualInterval':
        classifier = mapclassify.EqualInterval(gdf[column], k=k)
        class_labels = pd.Series(classifier.yb, index=gdf.index)
    elif method == 'NaturalBreaks':
        classifier = mapclassify.NaturalBreaks(gdf[column], k=k)
        class_labels = pd.Series(classifier.yb, index=gdf.index)
    elif method == 'EqualIntervalCustom':
        bins = [0, 25, 50, 75, 100]
        class_labels = pd.cut(
            gdf[column],
            bins=bins,
            labels=False,
            include_lowest=True
        )
    else:
        raise ValueError(f"Unsupported classification method: {method}")

    # Safe integer conversion
    gdf['class'] = class_labels.fillna(-1).astype(int)

    # Generate colors
    cmap = colormaps.get_cmap(palette)
    n_classes = int(gdf['class'].max()) + 1
    colors = [mcolors.to_hex(cmap(i / max(1, n_classes - 1))) for i in range(n_classes)]

    # Style function
    def style_function(feature):
        class_value = feature['properties']['class']
        if class_value < 0 or class_value >= len(colors):
            return {'fillOpacity': 0, 'weight': 0}
        return {
            'fillColor': colors[class_value],
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7
        }

    folium.GeoJson(
        gdf,  # GeoDataFrame directly
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['id', column])
    ).add_to(map_obj)

    return colors

# Create legend HTML
def create_legend_html(colors, column):
    intervals = [
        "0 – 25",
        "25 – 50",
        "50 – 75",
        "75 – 100"
    ]

    legend_html = f"""
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        width: 220px;
        border:2px solid grey;
        z-index:9999;
        font-size:14px;
        padding: 10px;
        background: rgba(255, 255, 255, 0.8);
    ">
    <b>{column}</b><br>
    """

    for color, label in zip(colors, intervals):
        legend_html += f"""
        <div style="display: flex; align-items: center; margin-bottom: 4px;">
            <div style="background:{color}; width:20px; height:20px; margin-right:6px;"></div>
            <span>{label}</span>
        </div>
        """

    legend_html += "</div>"
    return legend_html

# Generate the map and save
def generate_choropleth(gdf, column, method='Quantiles', palette='YlGn', selected_file=None, output_path='results/3_choropleth_maps/'):
    # Map
    centroid = calculate_centroid_of_union(gdf)
    m = folium.Map(location=centroid, zoom_start=13, control_scale=True)

    # Add choropleth
    colors = add_choropleth_manual(m, gdf, column, method=method, palette=palette)

    # Zoom to data
    bounds = gdf.total_bounds
    m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

    # Add legend
    legend_html = create_legend_html(colors, column)
    m.get_root().html.add_child(folium.Element(legend_html))

    filename = f'{column}_{study_area_name_only.replace(" ", "_")}_choropleth_{method}.html'
    save_path = os.path.join(output_path, filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save and display
    m.save(save_path)
    print(f"Choropleth map saved in: {save_path}")

    display(m)
    return m

# Generate map:
map_obj = generate_choropleth(
    gdf_osm,
    column='leisure_name_ratio',
    method='EqualIntervalCustom',
    palette='YlGn',
    selected_file=selected_file
)

Choropleth map saved in: results/3_choropleth_maps/leisure_name_ratio_Amsterdam_Centrum_West_choropleth_EqualIntervalCustom.html


In [98]:
# Checking the column used to calculate the quantiles
print(gdf_osm['building_name_ratio'].describe())

count     88.000000
mean       5.737455
std       15.823484
min        0.000000
25%        0.304878
50%        1.238687
75%        2.996233
max      100.000000
Name: building_name_ratio, dtype: float64


#### 4. Global Moran's Index

- The bellow cell code allows the user to select multiple variables from a GeoDataFrame and then analyze each one for spatial autocorrelation using Moran's I. The results, including the value of Moran's I and its p-value, are printed and visualized in a scatterplot for each selected variable. The selection widget's width is adjusted for better display.


- Statistical test

  - Null hypothesis represents the spatial independence of the data (Spatial distribution of the variable in question is random )
  - Confidence margin of 95% (p value < .05)

In [99]:
# FutureWarning ignore
warnings.simplefilter(action='ignore', category=FutureWarning)

# Function to perform Moran's I analysis
def analyze_morans_i(selected_vars):
    gdf = gdf_osm.reset_index(drop=True)

    # Create spatial weights
    w = weights.Queen.from_dataframe(gdf)
    w.transform = 'r'

    for var in selected_vars:
        y = gdf[var].values
        moran = Moran(y, w)

        # ---- Print results
        print(f"Moran's I for {var}: {moran.I:.4f}, p-value: {moran.p_sim:.4f}")

        # ---- Plot Moran scatterplot
        plot_moran(moran, zstandard=True, figsize=(10, 4))

        # ---- Define filename & full save path (consistent with choropleth)
        filename = f'{var}_{study_area_name_only.replace(" ", "_")}_MoransI.png'
        morans_output_path = 'results/4_Moran_Index'
        os.makedirs(morans_output_path, exist_ok=True)
        save_path = os.path.join(morans_output_path, filename)

        # ---- Save figure
        plt.savefig(save_path, dpi=300)
        plt.show()
        plt.close()

        print(f"Saved Moran scatterplot to: {save_path}")

# Button to perform the analysis and clear output
analyze_button = widgets.Button(description="Analyze Moran's I")
clear_output_button = widgets.Button(description="Clear Output")

# Create the selection widget for selecting multiple variables
select_variables = widgets.SelectMultiple(
    options=gdf_osm.columns,
    value=['leisure_name_ratio'],  # default value
    description='Variables:',
    disabled=False,
    layout={'width': '50%', 'height': '200px'}
)

# Function to handle button click event for analyzing Moran's I
def on_analyze_button_clicked(b):
    clear_output(wait=True)
    display_widgets()  # Redisplay widgets to maintain UI state
    analyze_morans_i(select_variables.value)

# Function to handle button click event for clearing the output
def on_clear_output_button_clicked(b):
    clear_output(wait=True)
    display_widgets()

# Function to display widgets with the appropriate layout
def display_widgets():
    display(VBox([
        select_variables,
        HBox([analyze_button, clear_output_button])
    ]))

# Connect the buttons to their respective event handlers
analyze_button.on_click(on_analyze_button_clicked)
clear_output_button.on_click(on_clear_output_button_clicked)

# Display the widgets initially
display_widgets()

#### 5. Hot Spot analysis using the Getis-Ord Gi statistic

In [100]:
# Hot Spot Analysis using Getis-Ord Gi* Statistic

warnings.simplefilter(action='ignore', category=FutureWarning)

# Calculate centroid of union of geometries
def calculate_centroid_of_union(gdf):
    union_poly = gdf.geometry.unary_union
    if isinstance(union_poly, MultiPolygon):
        union_poly = MultiPolygon(union_poly).convex_hull
    return [union_poly.centroid.y, union_poly.centroid.x]

# Color assignment based on Z-score
def color_producer(val):
    if val > 2.0:
        return 'red' # Vermelho
    elif 1.0 < val <= 2.0:
        return 'orange' # laranja
    elif -1.0 <= val <= 1.0:
        return 'darkgray' # cinza escuro
    elif -2.0 <= val < -1.0:
        return 'darkturquoise' # Ciano
    else:
        return 'blue' # Azul

# Calculate Getis-Ord Gi*
def calculate_getis_ord_gi(gdf, var_name):
    w = weights.distance.KNN.from_dataframe(gdf, k=8)
    w.transform = 'B'
    gi = G_Local(gdf[var_name], w, star=True)
    gdf[f"Gi_Z_{var_name}"] = gi.Zs
    return gdf

# Title HTML
def create_title_html(selected_var, study_area_name_only):
    title_html = f'''
    <div style="position: fixed;
    top: 10px; left: 50%; transform: translate(-50%, 0); width: auto;
    border:2px solid grey; z-index:9999; font-size:16px; font-weight: bold;
    background: rgba(255, 255, 255, 0.8); text-align: center; padding: 5px;">
    Getis-Ord Gi* Analysis for {selected_var} of {study_area_name_only}</div>
    '''
    return title_html

# Legend HTML
def create_legend_html():
    legend_html = '''
    <div style="position: fixed;
    bottom: 50px; left: 50px; width: 270px; height: auto;
    border:2px solid grey; z-index:9999; font-size:14px;
    background: rgba(255, 255, 255, 0.8);">
    &nbsp; <b>Legend</b> <br>
    &nbsp; Significant hot-spot (Z > 2.0) &nbsp; <i style="background:red;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Moderate hot-spot (1.0 < Z ≤ 2.0) &nbsp; <i style="background:orange;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Non-significant (-1.0 ≤ Z ≤ 1.0) &nbsp; <i style="background:darkgray;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Moderate cold-spot (-2.0 ≤ Z < -1.0) &nbsp; <i style="background:darkturquoise;width:10px;height:10px;display:inline-block;"></i><br>
    &nbsp; Significant cold-spot (Z < -2.0) &nbsp; <i style="background:blue;width:10px;height:10px;display:inline-block;"></i>
    </div>
    '''
    return legend_html

# Main function to analyze and generate map
def analyze_getis_ord_gi(selected_var):
    display("Wait: processing Getis-Ord Gi* analysis...")
    gdf_with_gi = calculate_getis_ord_gi(gdf_osm.copy(), selected_var)
    centroid_coords = calculate_centroid_of_union(gdf_with_gi)
    fig = Figure(width=1080, height=650)

    m = folium.Map(location=centroid_coords, zoom_start=14, control_scale=True)
    fig.add_child(m)

    folium.TileLayer(
        tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
        attr='CartoDB',
        name='CartoDB light'
    ).add_to(m)

    for _, row in gdf_with_gi.iterrows():
        centroid = row.geometry.centroid
        z_value = row[f"Gi_Z_{selected_var}"]
        folium.CircleMarker(
            location=[centroid.y, centroid.x],
            radius=3,
            color=color_producer(z_value),
            fill=True,
            fill_opacity=1
        ).add_to(m)

    def style_function(feature):
        value = feature['properties'][f"Gi_Z_{selected_var}"]
        return {
            'fillColor': color_producer(value),
            'color': 'gray',
            'weight': 1,
            'fillOpacity': 0.2,
            'lineOpacity': 0.7
        }

    folium.GeoJson(
        gdf_with_gi,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=[f"Gi_Z_{selected_var}"],
            aliases=["Z-score:"],
            localize=True
        )
    ).add_to(m)

    m.get_root().html.add_child(Element(create_legend_html()))
    m.get_root().html.add_child(Element(create_title_html(selected_var, study_area_name_only)))

    display(fig)

    output_filename = f'{selected_var}_{study_area_name_only.replace(" ", "_")}_getisord.html'
    getisord_output_path = 'results/5_getis_ord_maps/'
    save_path = os.path.join(getisord_output_path, output_filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    m.save(save_path)
    display(f"Map saved at: {save_path}")

# Widgets
select_variable_widget = widgets.Select(
    options=gdf_osm.columns,
    value='leisure_name_ratio', # default var
    description='Variable:',
    layout={'width': '30%', 'height': '150px'}
)

analyze_button = widgets.Button(description="Analyze and Show Map",
                                button_style='success')
clear_output_button = widgets.Button(description="Clear Output",
                                     button_style='danger')

box_layout = VBox([select_variable_widget, HBox([analyze_button, clear_output_button])])

def on_analyze_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)
    analyze_getis_ord_gi(select_variable_widget.value)

def on_clear_output_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

analyze_button.on_click(on_analyze_button_clicked)
clear_output_button.on_click(on_clear_output_button_clicked)

display(box_layout)

#### 6. Geographically Weighted Regression (GWR)

##### Reproject the GeoDataFrame to apply GWR

In [101]:
# Function to calculate the centroid of a GeoDataFrame

def get_utm_zone_crs(gdf):
    bounds = gdf.total_bounds
    bbox = box(bounds[0], bounds[1], bounds[2], bounds[3])
    centroid = bbox.centroid
    
    # Determine the UTM zone from the centroid's longitude
    utm_zone = int((centroid.x + 180) / 6) + 1
    hemisphere = 'north' if centroid.y > 0 else 'south'

    # Create a UTM CRS based on the centroid location
    utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +{hemisphere} +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    
    return utm_crs

# Obtain the appropriate UTM CRS
utm_crs = get_utm_zone_crs(gdf_osm)

# Project the GeoDataFrame to the detected UTM CRS
gdf_osm_projected = gdf_osm.to_crs(utm_crs.to_string())

gdf_osm_projected.crs

<Derived Projected CRS: +proj=utm +zone=31 +north +ellps=WGS84 +datum=WGS8 ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 31N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

##### Applying VIF (Variance Inflation Factor) to identify multicollinearity

In [102]:
# Applying VIF (Variance Inflation Factor) to identify multicollinearity
# among independent variables in regression models

# --- Setup ---

# Output directories
vif_output_dir = 'results/6_gwr/vif/'
corr_output_dir = 'results/6_gwr/vif/correlation_matrix/'

# Ensure directories exist
os.makedirs(vif_output_dir, exist_ok=True)
os.makedirs(corr_output_dir, exist_ok=True)

warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Widgets ---
select_x_variables_widget = SelectMultiple(
    options=[col for col in gdf_osm_projected.columns if col != 'geometry'],
    description='Independent Variables:',
    disabled=False,
    layout=Layout(width='40%', height='200px')
)

vif_button = Button(description="Calculate VIF")
correlation_button = Button(description="Show Correlation Matrix")
clear_button = Button(description="Clear Results")

box_layout = VBox([
    select_x_variables_widget,
    HBox([vif_button, correlation_button, clear_button])
])

display(box_layout)

# Global variable to store detected OSM tag
detected_tag = 'unknown'

# --- Function to calculate VIF, save CSV, and show download link ---
def calculate_vif(b):
    global detected_tag # Declare usage of the detected_tag as global variable
    clear_output(wait=True)
    display(box_layout)
    selected_vars = list(select_x_variables_widget.value)

    if len(selected_vars) < 2:
        print("Please select at least two variables to calculate VIF.")
        return

    variables = gdf_osm_projected[selected_vars]

    vif_data = pd.DataFrame({
        "feature": variables.columns,
        "VIF": [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
    })

    display(vif_data)

    # Detect OSM Tag dynamically
    tags_in_selection = [col.split('_')[0] for col in selected_vars if '_' in col]
    if tags_in_selection:
        detected_tag = tags_in_selection[0]
    else:
        detected_tag = 'unknown'

    # Save VIF to CSV
    save_path = os.path.join(vif_output_dir, f'vif_{detected_tag}_{study_area_name_only.replace(" ", "_")}.csv')
    vif_data.to_csv(save_path, index=False)
    print(f"\u2705 VIF saved at: {save_path}")

    # Provide download link
    display(FileLink(save_path, result_html_prefix="Click to download VIF file: "))

# --- Function to show correlation matrix, save PNG, and show download link ---
def show_correlation_matrix(b):
    clear_output(wait=True)
    display(box_layout)
    selected_vars = list(select_x_variables_widget.value)

    if len(selected_vars) < 2:
        print("Please select at least two variables to view the correlation matrix.")
        return

    variables = gdf_osm_projected[selected_vars]
    corr_matrix = variables.corr()

    # Plot and save
    plt.figure(figsize=(8,6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix of Independet Variables')

    # Add study area name

    corr_save_path = os.path.join(corr_output_dir, f'correlation_matrix_{detected_tag}_{study_area_name_only.replace(" ", "_")}.png')
    plt.savefig(corr_save_path, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"\u2705 Correlation matrix saved at: {corr_save_path}")
    display(FileLink(corr_save_path, result_html_prefix="Click to download correlation matrix: "))

# --- Function to clear the output ---
def on_clear_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

# --- Event bindings ---
vif_button.on_click(calculate_vif)
correlation_button.on_click(show_correlation_matrix)
clear_button.on_click(on_clear_button_clicked)

##### Check explained variance ratio

In [103]:
# Checking the explained variance

features = [col for col in gdf_osm_projected.columns if col != 'geometry' and col not in ['leisure_name_ratio', 'building_name_ratio', 'amenity_name_ratio']]
select_features_widget = SelectMultiple(
    options=features,
    value=[features[0]],
    description='Variables:',
    disabled=False,
    layout=Layout(width='50%', height='120px')
)

plot_button = Button(description="Plot Explained Variance")
clear_button = Button(description="Clear Results")

# Layout setup
box_layout = VBox([select_features_widget,
                   HBox([plot_button, clear_button])
                   ])
display(box_layout)

# Function to plot explained variance
def plot_variance(b):
    clear_output(wait=True)
    display(box_layout)
    selected_features = list(select_features_widget.value)
        
    # Standardizing the selected features
    x = gdf_osm_projected[selected_features]
    x_scaled = StandardScaler().fit_transform(x)
        
     # Applying PCA and capturing the explained variance
    pca = PCA()
    pca.fit(x_scaled)
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
        
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
    plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

# Function to clear results
def clear_results(b):
    clear_output(wait=True)
    display(box_layout)

# Binding the buttons to their respective functions
plot_button.on_click(plot_variance)
clear_button.on_click(clear_results)

##### Perform Geographically Weighted Regression (GWR)

In [104]:
# Perform Geographically Weighted Regression (GWR) with PCA

gdf_osm_projected = gdf_osm_projected.copy()

# --- Setup Directories ---

gwr_output_dir = 'results/6_gwr/gwr_pca/'

os.makedirs(gwr_output_dir, exist_ok=True)

warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Widgets ---
select_y_variable_widget = Dropdown(
    options=[col for col in gdf_osm_projected.columns if col != 'geometry'],
    value='leisure_name_ratio',  # Default suggested
    description='Dependent Variable:',
    disabled=False
)

select_x_variables_widget = SelectMultiple(
    options=[col for col in gdf_osm_projected.columns if col != 'geometry'],
    description='Independent Variables:',
    disabled=False,
    layout=Layout(width='50%', min_width='300px', height='200px')
)

analyze_button = Button(description="Perform GWR with PCA", layout=Layout(width='auto', min_width='200px'))
clear_button = Button(description="Clear Results", layout=Layout(width='auto', min_width='120px'))

box_layout = VBox([
    select_y_variable_widget,
    select_x_variables_widget,
    HBox([analyze_button, clear_button], layout=Layout(margin='10px 0'))
], layout=Layout(margin='0 0 10px 0'))

display(box_layout)

# --- Main Analysis Function ---
def on_analyze_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)
    display(" Processing GWR with PCA, please wait...")

    selected_y_var = select_y_variable_widget.value
    selected_x_vars = list(select_x_variables_widget.value)

    if len(selected_x_vars) < 2:
        display("Please select at least two independent variables.")
        return

    # Detect TAG from independent variables
    tags_in_selection = [col.split('_')[0] for col in selected_x_vars if '_' in col]
    detected_tag = tags_in_selection[0] if tags_in_selection else 'unknown'

    study_area_name_only = extract_study_area_name(selected_file)

    # --- Standardize Features ---
    features = gdf_osm_projected[selected_x_vars]
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # --- Perform PCA ---
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(features_scaled)
    principal_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
    principal_df['DependentVar'] = gdf_osm_projected[selected_y_var]

    # --- Save PCA Loadings ---
    pca_loadings = pd.DataFrame(
        pca.components_,
        columns=selected_x_vars,
        index=['PC1', 'PC2']
    )
    display(pca_loadings)

    loadings_save_path = os.path.join(
        gwr_output_dir, f'pca_loadings_{detected_tag}_{study_area_name_only.replace(" ", "_")}.csv'
    )
    pca_loadings.to_csv(loadings_save_path)
    print(f"PCA loadings saved at: {loadings_save_path}")

    # --- Prepare coordinates and target ---
    coords = np.array(list(zip(gdf_osm_projected.geometry.centroid.x, gdf_osm_projected.geometry.centroid.y)))
    y = principal_df['DependentVar'].values.reshape((-1, 1))
    X = principal_df[['PC1', 'PC2']].values

    try:
        gwr_selector = Sel_BW(coords, y, X, kernel='bisquare', fixed=False)
        bw = gwr_selector.search()
        gwr_model = GWR(coords, y, X, bw).fit()

        # --- Store GWR results with TAG in column names ---
        gdf_osm_projected[f'Local_R2_{detected_tag}'] = gwr_model.localR2
        gdf_osm_projected[f'PC1_coef_{detected_tag}'] = gwr_model.params[:, 1]
        gdf_osm_projected[f'PC2_coef_{detected_tag}'] = gwr_model.params[:, 2]

        # --- Plot Results ---
        fig, axs = plt.subplots(1, 3, figsize=(18, 5))

        scatter1 = axs[0].scatter(coords[:, 0], coords[:, 1], c=gdf_osm_projected[f'Local_R2_{detected_tag}'], cmap='viridis_r')
        axs[0].set_title(f'Local R²')
        plt.colorbar(scatter1, ax=axs[0])

        scatter2 = axs[1].scatter(coords[:, 0], coords[:, 1], c=gdf_osm_projected[f'PC1_coef_{detected_tag}'], cmap='coolwarm')
        axs[1].set_title(f'GWR Local Regression Coefficient - PC1')
        plt.colorbar(scatter2, ax=axs[1])

        scatter3 = axs[2].scatter(coords[:, 0], coords[:, 1], c=gdf_osm_projected[f'PC2_coef_{detected_tag}'], cmap='coolwarm')
        axs[2].set_title(f'GWR Local Regression Coefficient - PC2')
        plt.colorbar(scatter3, ax=axs[2])

        plt.tight_layout()

        map_save_path = os.path.join(
            gwr_output_dir, f'gwr_pca_maps_{detected_tag}_{study_area_name_only.replace(" ", "_")}.png'
        )
        plt.savefig(map_save_path, dpi=300, bbox_inches='tight')
        plt.show()

        print(f"GWR maps saved at: {map_save_path}")

    except Exception as e:
        # Catch any GWR failure including LinAlgError
        print(f"GWR could not be computed: {e}. Skipping this run.")


# --- Clear Function ---
def on_clear_button_clicked(b):
    clear_output(wait=True)
    display(box_layout)

# --- Bind Buttons ---
analyze_button.on_click(on_analyze_button_clicked)
clear_button.on_click(on_clear_button_clicked)

In [105]:
y = gdf_osm_projected[select_y_variable_widget.value]

print("Dependent variable summary:")
print(y.describe())

print("\nZero values:", (y == 0).sum())
print("Near-zero values:", (np.abs(y) < 1e-6).sum())
print("Unique values:", y.nunique())

Dependent variable summary:
count     88.000000
mean      63.623105
std       28.267056
min        0.000000
25%       48.409091
50%       70.370370
75%       86.363636
max      100.000000
Name: amenity_name_ratio, dtype: float64

Zero values: 5
Near-zero values: 5
Unique values: 59


#### 7. Saving the Final GeoJSON

In [106]:
# Save the final GeoDataFrame as GeoJSON
gdf_osm_map = gdf_osm_projected.to_crs(epsg=4326)
gdf_osm_map.to_file(os.path.join(gwr_output_dir, f'gdf_osm_projected_gwr_pca_results_{study_area_name_only.replace(" ", "_")}.geojson'), driver='GeoJSON')
print(f'GeoJSON saved at: {gwr_output_dir}')

GeoJSON saved at: results/6_gwr/gwr_pca/
